**OVERVIEW**

**Objective**

The objective of this project is to increase CTR on a marketing email dataset.

**Dataset**

The dataset consists of 99,950 emails, with fields WRT
1. email metadata
2. user's metadata
3. clicked / not

_________

**SETUP**

Import

In [ ]:
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

Set configurations

In [ ]:
pd.set_option('display.max_columns', 20)
pd.set_option('display.width', 350)

np.random.seed(4684)

_________

**BUILD MODEL**

**Read data**

In [ ]:
data = pd.read_csv('https://drive.google.com/uc?export=download&id=1PXjbqSMu__d_ppEv92i_Gnx3kKgfvhFk')

**Bin data**

Bin data and speed up execution.

Bin data by hour

The dataset is binned as follows:

* 1 AM - 5 AM
* 5 AM - 1 PM
* 1 PM - 9 PM
* 9 PM - 2 AM

In [ ]:
data['hour_binned'] = pd.cut(data['hour'], bins=[1, 5, 13, 21, 24], include_lowest=True, labels=['night', 'morning', 'afternoon', 'night2'])
data['hour_binned'] = data['hour_binned'].replace('night2', 'night').cat.remove_unused_categories()
print(data)


       email_id   email_text email_version  hour    weekday user_country  user_past_purchases  clicked hour_binned
0             8  short_email       generic     9   Thursday           US                    3        0     morning
1            33   long_email  personalized     6     Monday           US                    0        0     morning
2            46  short_email       generic    14    Tuesday           US                    3        0   afternoon
3            49   long_email  personalized    11   Thursday           US                   10        0     morning
4            65  short_email       generic     8  Wednesday           UK                    3        0     morning
...         ...          ...           ...   ...        ...          ...                  ...      ...         ...
99945    999969  short_email       generic    21   Thursday           US                    0        0   afternoon
99946    999972   long_email  personalized     6    Tuesday           US        

Bin data by purchases

The dataset is binned as follows:

*   [0, 1) purchases
*   [1, 4) purchases
*   [4, 8) purchases
*   [8, 23) purchases



In [ ]:
data['purchase_binned'] = pd.cut(data['user_past_purchases'], bins=[0,1, 4, 8, 23], include_lowest=True, right=False, labels=['None', 'Low', 'Medium', 'High'])
print(data)

       email_id   email_text email_version  hour    weekday user_country  user_past_purchases  clicked hour_binned purchase_binned
0             8  short_email       generic     9   Thursday           US                    3        0     morning             Low
1            33   long_email  personalized     6     Monday           US                    0        0     morning            None
2            46  short_email       generic    14    Tuesday           US                    3        0   afternoon             Low
3            49   long_email  personalized    11   Thursday           US                   10        0     morning            High
4            65  short_email       generic     8  Wednesday           UK                    3        0     morning             Low
...         ...          ...           ...   ...        ...          ...                  ...      ...         ...             ...
99945    999969  short_email       generic    21   Thursday           US           

Prepare data for modelling

In [ ]:
data_dummy = pd.get_dummies(data, drop_first=True).drop(['email_id', 'hour', 'user_past_purchases'], axis=1)

train, test = train_test_split(data_dummy, test_size = 0.34)

Build model and fit data

Use a Random Forest of 50 trees

In [ ]:
train, test = train_test_split(data_dummy, test_size = 0.34)
rf = RandomForestClassifier(class_weight={0:0.05,1:0.95}, n_estimators=50, oob_score=True)
rf.fit(train.drop('clicked', axis=1), train['clicked'])


RandomForestClassifier(class_weight={0: 0.05, 1: 0.95}, n_estimators=50,
                       oob_score=True)

Print OOB confusion matrix

In [ ]:
conf_matrix = confusion_matrix(train['clicked'], rf.oob_decision_function_[:,1].round(), labels=[0, 1])

print(pd.DataFrame(conf_matrix))

       0     1
0  58538  6069
1   1073   287


In [ ]:
# from pprint import pprint
# pprint(vars(rf))

______________________________________

**Predict click-through-rate for each segment**

Remove classification label and drop duplicates

In [ ]:
data_unique = data_dummy.drop(['clicked'], axis=1)
data_unique = data_unique.drop_duplicates()
# print(data_unique)

Predict & add to dataset

In [ ]:
predictions = rf.predict_proba(data_unique)
# prediction -> [P(class_0), P(class_1)]

data_unique['prediction'] = [x[1] for x in predictions]
data_unique

,email_text_short_email,email_version_personalized,weekday_Monday,weekday_Saturday,weekday_Sunday,weekday_Thursday,weekday_Tuesday,weekday_Wednesday,user_country_FR,user_country_UK,user_country_US,hour_binned_morning,hour_binned_afternoon,purchase_binned_Low,purchase_binned_Medium,purchase_binned_High,prediction
0,1,0,0,0,0,1,0,0,0,0,1,1,0,1,0,0,0.330359
1,0,1,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0.000000
2,1,0,0,0,0,0,1,0,0,0,1,0,1,1,0,0,0.000000
3,0,1,0,0,0,1,0,0,0,0,1,1,0,0,0,1,0.704885
4,1,0,0,0,0,0,0,1,0,1,0,1,0,1,0,0,0.379019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45505,1,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0.000000
48374,0,1,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0.000000
52250,1,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0.000000
57213,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0.000000


______________________________________

**Identify the best email characteristics for each user**

1. Sort the records by descending probability of clicking
2. Remove duplicate entries with the same parameters to find the combination of user characteristics that has max. P(click)

In [ ]:
data_unique = data_unique.sort_values('prediction', ascending=False)
  
subsets = ['user_country_FR', 'user_country_UK', 'user_country_US', 'purchase_binned_Low', 'purchase_binned_Medium', 'purchase_binned_High']
best_segment = data_unique.drop_duplicates(subset=subsets).copy()

best_segment

,email_text_short_email,email_version_personalized,weekday_Monday,weekday_Saturday,weekday_Sunday,weekday_Thursday,weekday_Tuesday,weekday_Wednesday,user_country_FR,user_country_UK,user_country_US,hour_binned_morning,hour_binned_afternoon,purchase_binned_Low,purchase_binned_Medium,purchase_binned_High,prediction
6151,1,1,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0.753177
1359,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0.742606
55,1,1,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0.709333
1333,1,1,0,0,0,0,0,1,0,1,0,1,0,0,1,0,0.659346
22200,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0.632272
10352,1,1,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0.593346
3390,1,0,0,1,0,0,0,0,0,1,0,0,1,1,0,0,0.593037
138,1,1,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0.562108
144,1,1,0,0,0,0,1,0,0,0,1,0,1,0,1,0,0.551692
647,1,1,0,0,0,0,1,0,0,0,1,0,1,1,0,0,0.541721


Making dataset human-readable

In [ ]:
#Country
best_segment['user_country'] = np.where(best_segment['user_country_UK'] == 1, "UK", 
                                   np.where(best_segment['user_country_US'] == 1, "US", 
                                      np.where(best_segment['user_country_FR'] == 1, "FR",
                                     "ES"
)))
best_segment = best_segment.drop([e for e in list(data_unique) if e.startswith('user_country_')], axis=1)
  
#Number_purchases
best_segment['purchase_binned'] = np.where(best_segment['purchase_binned_High'] == 1, "High", 
                                   np.where(best_segment['purchase_binned_Medium'] == 1, "Medium", 
                                    np.where(best_segment['purchase_binned_Low'] == 1, "Low",
                                     "None"
)))
best_segment = best_segment.drop([e for e in list(data_unique) if e.startswith('purchase_binned_')], axis=1)
  
#Email Text
best_segment['email_text'] = np.where(best_segment['email_text_short_email'] == 1, "short_email", "long_email")
best_segment = best_segment.drop('email_text_short_email', axis=1)
  
#Email version
best_segment['email_version'] = np.where(best_segment['email_version_personalized'] == 1, "personalized", "generic")
best_segment = best_segment.drop('email_version_personalized', axis=1)
  
#Weekday
best_segment['weekday'] = np.where(best_segment['weekday_Monday'] == 1, "Monday", 
                                    np.where(best_segment['weekday_Saturday'] == 1, "Saturday", 
                                       np.where(best_segment['weekday_Sunday'] == 1, "Sunday",
                                          np.where(best_segment['weekday_Thursday'] == 1, "Thursday", 
                                              np.where(best_segment['weekday_Tuesday'] == 1, "Tuesday",
                                                   np.where(best_segment['weekday_Wednesday'] == 1, "Wednesday",
                                                      "Friday"
))))))
best_segment = best_segment.drop([e for e in list(data_unique) if e.startswith('weekday_')], axis=1)      
  
#Hour
best_segment['hour_binned'] = np.where(best_segment['hour_binned_afternoon'] == 1, "afternoon", 
                                   np.where(best_segment['hour_binned_morning'] == 1, "morning", 
                                     "night"
))
best_segment = best_segment.drop([e for e in list(data_unique) if e.startswith('hour_binned_')], axis=1) 
best_segment

,prediction,user_country,purchase_binned,email_text,email_version,weekday,hour_binned
6151,0.753177,UK,High,short_email,personalized,Monday,night
1359,0.742606,US,High,long_email,personalized,Wednesday,night
55,0.709333,ES,High,short_email,personalized,Tuesday,morning
1333,0.659346,UK,Medium,short_email,personalized,Wednesday,morning
22200,0.632272,FR,High,short_email,generic,Monday,night
10352,0.593346,ES,Medium,short_email,personalized,Monday,afternoon
3390,0.593037,UK,Low,short_email,generic,Saturday,afternoon
138,0.562108,FR,Medium,short_email,personalized,Monday,night
144,0.551692,US,Medium,short_email,personalized,Tuesday,afternoon
647,0.541721,US,Low,short_email,personalized,Tuesday,afternoon


* Now we have a model that returns the best email strategy for each user combination

* Caveat: even the best email strategy has low probabilities for users with no purchases, regardless of country. 

* We won’t get those people to convert just through emails. 

* We may have to make product changes for that. After all, only good segments are marketing opportunities. 

Sample: combinations of UK user characteristics with highest probability of clicking

In [ ]:
best_segment.loc[data['user_country'] == 'UK']

,prediction,user_country,purchase_binned,email_text,email_version,weekday,hour_binned
6151,0.753177,UK,High,short_email,personalized,Monday,night
1333,0.659346,UK,Medium,short_email,personalized,Wednesday,morning
3390,0.593037,UK,Low,short_email,generic,Saturday,afternoon
2850,0.000000,UK,None,long_email,personalized,Thursday,night


_________

**Estimate A/B test gains**

* We have a model to send personalized emails.

* We now have to test it.
* In order to test, I have to run the model on a randomized fraction of users 
* Then, I would compare its results with the current email model.

* In order to run the test though, the product manager has to be convinced that it makes sense to run the test from a cost-opportunity standpoint.
* The best way to do that is giving them an estimate of by how much we think we could potentially increase click-rate.
* That way they can figure out whether it makes sense.

* Since we know the predicted probability for each group, we can just estimate the weighted average to estimate the final overall click-rate. 


* Caveat: the predicted probability from the model is insufficient. 
* Reason: model has pretty high class 1 error. 

* Hence, we need to to adjust the predicted probabilities after taking into account the model expected error.

Count by group

In [ ]:
country_purchase_count = data[['user_country','purchase_binned']].groupby(['user_country','purchase_binned']).size()
print(country_purchase_count)

user_country  purchase_binned
ES            None                1368
              Low                 3785
              Medium              3389
              High                1422
FR            None                1341
              Low                 3890
              Medium              3314
              High                1444
UK            None                2791
              Low                 7803
              Medium              6622
              High                2712
US            None                8372
              Low                23364
              Medium             20008
              High                8325
dtype: int64


In [ ]:
country_purchase_count.reset_index(name='counts')

,user_country,purchase_binned,counts
0,ES,None,1368
1,ES,Low,3785
2,ES,Medium,3389
3,ES,High,1422
4,FR,None,1341
5,FR,Low,3890
6,FR,Medium,3314
7,FR,High,1444
8,UK,None,2791
9,UK,Low,7803


In [ ]:
segments = ['user_country','purchase_binned']

count_segment = data[segments].groupby(segments).size().reset_index(name='counts')
count_segment['weight'] = count_segment['counts'].div(count_segment['counts'].sum())

count_segment

,user_country,purchase_binned,counts,weight
0,ES,None,1368,0.013687
1,ES,Low,3785,0.037869
2,ES,Medium,3389,0.033907
3,ES,High,1422,0.014227
4,FR,None,1341,0.013417
5,FR,Low,3890,0.038919
6,FR,Medium,3314,0.033157
7,FR,High,1444,0.014447
8,UK,None,2791,0.027924
9,UK,Low,7803,0.078069


In [ ]:
best_segment = pd.merge(best_segment, count_segment).sort_values('prediction', ascending=False)
best_segment

,prediction,user_country,purchase_binned,email_text,email_version,weekday,hour_binned,counts,weight
0,0.753177,UK,High,short_email,personalized,Monday,night,2712,0.027134
1,0.742606,US,High,long_email,personalized,Wednesday,night,8325,0.083292
2,0.709333,ES,High,short_email,personalized,Tuesday,morning,1422,0.014227
3,0.659346,UK,Medium,short_email,personalized,Wednesday,morning,6622,0.066253
4,0.632272,FR,High,short_email,generic,Monday,night,1444,0.014447
5,0.593346,ES,Medium,short_email,personalized,Monday,afternoon,3389,0.033907
6,0.593037,UK,Low,short_email,generic,Saturday,afternoon,7803,0.078069
7,0.562108,FR,Medium,short_email,personalized,Monday,night,3314,0.033157
8,0.551692,US,Medium,short_email,personalized,Tuesday,afternoon,20008,0.200180
9,0.541721,US,Low,short_email,personalized,Tuesday,afternoon,23364,0.233757


Adding Class 0 and Class 1 errors to the dataset

In [ ]:
conf_matrix = pd.DataFrame(confusion_matrix(test['clicked'], rf.predict(test.drop('clicked', axis=1)), labels=[0, 1]))
conf_matrix

,0,1
0,30265,3009
1,535,174


Positive Predictive Value (for class 1)

In [ ]:
true_positives = conf_matrix.loc[1,1]
false_positives = conf_matrix.loc[0,1]
ppv = true_positives / (true_positives + false_positives)

ppv

0.05466540999057493

False omission rate = 1 - precision (for class 0)


In [ ]:
true_negatives = conf_matrix.loc[0,0]
false_negatives = conf_matrix.loc[1,0]
false_omission_rate = false_negatives / (false_negatives + true_negatives)
false_omission_rate

0.01737012987012987

Calculating adjusted prediction

In [ ]:
best_segment['adjusted_prediction'] = best_segment['prediction'] * ppv + (1-best_segment['prediction']) * false_omission_rate
best_segment

,prediction,user_country,purchase_binned,email_text,email_version,weekday,hour_binned,counts,weight,adjusted_prediction
0,0.753177,UK,High,short_email,personalized,Monday,night,2712,0.027134,0.045460
1,0.742606,US,High,long_email,personalized,Wednesday,night,8325,0.083292,0.045066
2,0.709333,ES,High,short_email,personalized,Tuesday,morning,1422,0.014227,0.043825
3,0.659346,UK,Medium,short_email,personalized,Wednesday,morning,6622,0.066253,0.041961
4,0.632272,FR,High,short_email,generic,Monday,night,1444,0.014447,0.040951
5,0.593346,ES,Medium,short_email,personalized,Monday,afternoon,3389,0.033907,0.039499
6,0.593037,UK,Low,short_email,generic,Saturday,afternoon,7803,0.078069,0.039488
7,0.562108,FR,Medium,short_email,personalized,Monday,night,3314,0.033157,0.038334
8,0.551692,US,Medium,short_email,personalized,Tuesday,afternoon,20008,0.200180,0.037946
9,0.541721,US,Low,short_email,personalized,Tuesday,afternoon,23364,0.233757,0.037574


In [ ]:
CTR_comparison = pd.DataFrame( {'predicted_click_rate':[(best_segment['adjusted_prediction']*best_segment['weight']).sum()], 'old_click_rate':[data['clicked'].mean()] })
CTR_comparison

,predicted_click_rate,old_click_rate
0,0.036441,0.0207
